可以通过 for...in... 循环来遍历可迭代对象，如列表、元组、字符串、字典等。

In [ ]:
from typing import Iterable, Iterator

# common methods

In [1]:
# 可迭代对象的共同属性
def common_property(*objs):
    """计算对象之间的共同属性"""
    assert len(objs) > 0, "至少需要两个对象"
    attrs = set(dir(objs[0]))
    for obj in objs[1:]:
        # 两个对象共同拥有的属性
        attrs &= set(dir(obj))
    # 除去 object 自带的属性
    attrs -= set(dir(object))
    return attrs


# 可迭代对象

In [ ]:
# 可迭代对象共同属性
iterables = ["hello", [1, 2, 3], (1, 2, 3), {1, 2, 3}, {1: 2, 3: 4}, open("iterable1.ipynb")]
common_property(*iterables)
# 对于可迭代对象，for...in...循环会自动调用 __iter__ 方法，该方法返回一个迭代器对象，然后通过迭代器对象来获取元素。

{'__iter__'}

In [3]:
# __iter__ 方法, 对应的调用方法就是 iter() 函数, 该函数返回一个迭代器对象。

In [ ]:
for iterable in iterables:
    print(iter(iterable))

<_io.TextIOWrapper name='iterable1.py' mode='r' encoding='utf-8'>


## 用例，定义一个容器类的可迭代对象，为该对象实现 `__iter__()` 方法，返回一个生成器对象。

In [ ]:
class MyCustomIterator:
    def __init__(self, data):
        self.data = data
        self.index = -1

    def __iter__(self):
        return self

    def __next__(self):
        self.index += 1
        if self.index < self.data.size:
            return self.data.get(self.index)
        else:
            raise StopIteration

# 可迭代数据类
class MyCustonData:
    ...

    @property
    def size(self):
        return self.size

    def get(self, index):
        return self.data[index]

    def __iter__(self):
        return MyCustomIterator(self)

In [ ]:
# 使用 yield 关键字简化上述步骤
# 可迭代数据类
class MyCustonData1:
    ...

    @property
    def size(self):
        return self.size

    def get(self, index):
        return self.data[index]

    def __iter__(self):
        # index 必须是局部变量
        index = 0
        while index < self.size:
            yield self.data[index]
            index += 1

# 迭代器对象

In [ ]:
# 迭代器对象共同属性
iterators = [iter(iterable) for iterable in iterables]
print(common_property(*iterators))

{'__iter__', '__next__'}


`__next__` 方法通过调用 `next()` 方法来获取下一个元素，如果没有下一个元素，则抛出 `StopIteration` 异常。

`__iter__` 方法返回一个迭代器对象，用于 `for` 循环。

In [2]:

actions = ["点赞", "评论", "收藏"]
action_iterator = iter(actions)
action_iterator

In [3]:
next(action_iterator)

'点赞'

In [4]:
next(action_iterator)

'评论'

In [5]:
next(action_iterator)

'收藏'

In [ ]:
next(action_iterator)
# StopIteration

StopIteration: 

迭代的3个步骤

1. 调用 `iter(iterable)` 方法，将可迭代对象转换为迭代器。
2. 多次使用 `next(iterator)` 方法，获取迭代器的下一个元素。
3. 当迭代器的元素耗尽时，抛出 `StopIteration` 异常，表示迭代结束。

In [8]:
# 通过 while 循环实现迭代器
action_iterator = iter(actions)
while True:
    try:
        action = next(action_iterator)
        print(action)
    except StopIteration:
        break

点赞
评论
收藏


# 自定义迭代器


迭代器基本功能

- 初始化时，传入一个可迭代对象
- 要初始化迭代进度
- 每次迭代时，即调用 `next()` 方法：
  - 如果有元素可以迭代，返回本轮迭代元素，同时更新迭代进度
  - 如果没有元素可以迭代，抛出 `StopIteration` 异常

再添加一点额外的逻辑：

- 设置一个黑名单，如果当前数据在黑名单内，则跳过该元素
- 将某些符合条件的数据 `*2` 之后再返回

## 迭代器协议

Python 文档中明确指出，迭代器必须同时实现 `__next__` 和 `__iter__` 方法。

根据这个协议，迭代器 **必须** 是可迭代的，**迭代器是一种可迭代对象**。

所有迭代器的 `__iter__` 方法必须返回自身，而 `__next__` 方法必须返回下一个元素。

In [17]:
BLACK_LIST = ["白嫖", "取关"]
class ShuziIterator:
    def __init__(self, actions) -> None:
        self.actions = actions
        self.index = 0

    def __next__(self):
        while self.index < len(self.actions):
            action = self.actions[self.index]
            self.index += 1
            if action in BLACK_LIST:
                continue
            elif "币" in action:
                return action * 2
            else:
                return action
        raise StopIteration

    # 在 for 循环中使用
    def __iter__(self):
        return self


actions = ["点赞", "收藏", "转发", "评论", "投币", "白嫖", "取关"]
sz_iterator = ShuziIterator(actions)
while True:
    try:
        print(next(sz_iterator))
    except StopIteration:
        break

点赞
收藏
转发
评论
投币投币


In [ ]:
# ShuziIterator 要实现 __iter__ 方法, 才能使用 for 循环遍历
sz_iterator = ShuziIterator(actions)
for item in sz_iterator:
    print(item)

点赞
收藏
转发
评论
投币投币


# 迭代器意义

- 通过统一 `next()` 方法获取数据，可以屏蔽底层不同的数据读取方式，简化编程
- 容器类的数据结构之关系数据的静态存储，每次迭代都需要额外的迭代器对象专门负责记录迭代过程中的状态信息
- 迭代器是为了让数据结构快捷的遍历而定义的辅助对象
- 很多个迭代器串联起来，形成一个数据处理管道
- 在这个管道中，每次一只通过一份数据，避免一次性加载所有数据
- 迭代器也不仅仅是按顺序返回数据那么简单，可以承担处理数据的责任
  - 迭代器可以用来实现数据流的处理，比如数据压缩、加密、解密、过滤、排序等等，例如 `ShuziIterator` 实现了数据过滤的功能
- 当通过迭代器获取数据的时候，原理的数据存储，不关心底层数据是怎么存储的
- 数据生成, 下面生成器的例子就是，每次实时生成数据，不占用内存空间

In [21]:
from random import random

class Random:
    def __iter__(self):
        return self

    def __next__(self):
        return random()

In [22]:
random1 = Random()
for i, r in enumerate(random1):
    print(r)
    if i == 10:
        break

0.27645103060096377
0.2605600226315695
0.47010276552565655
0.20827978083608867
0.5833221624875383
0.2727301822066459
0.9696029573450561
0.9865410147876872
0.9091535977689023
0.853023640820604
0.9576099972089506
